# Improve SS Performance - eliminate negative profit months

## Potential Problems From Visual Observations

* on winner and loosers there are buys and sells at levels of resistance that would loose
* there are multiple buys and sells on a flat line

## Improvements to try

1. if the next bars close is = to the last don't buy/sell (unsure this may reduce profit factor, the duplications are useful but maybe less so on the flat line) -- epic fail complete negative returns
2. near needs to be one bar away -- it already is < 2 is 1 bar away



3. there should be a buy if closes outside the bottom bb and bb pointing down
4. there should be a sell if closes outside the to bb and bb pointing up


5. reduce the average adr from 7 days to 3 days -- epic fail, complete negative returns

In [ ]:
import numpy as np
import warnings
import pandas as pd
import pyperclip as clip
import ta
warnings.filterwarnings('ignore', category=FutureWarning)

128800 rows at 1m

102909 rows at 5min

In [ ]:
output_path = '../tmp-data/ss_strategy_1min_7d_avg_adr_range_bars.check3.csv'

In [ ]:
csv_paths = ['../data/BTCUSDT_1m_3m_back_from_2023-03-14.csv', '../data/BTCUSDT_1m_3m_back_from_2022-11-15.csv', '../data/BTCUSDT_1m_3m_back_from_2022-07-19.csv']
df = pd.DataFrame()
for csv_path in csv_paths:
    df = pd.concat([df, pd.read_csv(csv_path, usecols=['timestamp', 'open', 'high', 'low', 'close', 'volume'])])

df['date'] = pd.to_datetime(df['timestamp']).dt.date 
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
# df = df.resample('5T').agg({
#     'date': 'first',
#     'open': 'first',
#     'high': 'max',
#     'low': 'min',
#     'close': 'last'
# })
df.rename(columns={'close': 'Close', 'open': 'Open', 'high': 'High', 'low': 'Low'}, inplace=True)
df

In [ ]:
def adr(df: pd.DataFrame) -> float:
    daily_high_low = df.groupby('date')['High', 'Low'].agg(['max', 'min'])
    daily_high_low['adr'] = daily_high_low[('High', 'max')] - daily_high_low[('Low', 'min')]
    return np.mean(daily_high_low['adr'])

def relative_adr_range_size(df_in: pd.DataFrame, resample_arg: str = 'W') -> str:
    # resample the dataframe into monthly periods
    groups = df_in.resample(resample_arg)
    df_out = pd.DataFrame()
    for _, group in groups:
        week_day_seg = group.copy()
        average_adr = adr(week_day_seg)
        week_day_seg['average_adr'] = average_adr
        df_out = pd.concat([df_out, week_day_seg])
    return df_out   

In [ ]:
df2 = relative_adr_range_size(df.copy())
df2

In [ ]:
def create_range_bar_df(df: pd.DataFrame):
        range_bars = []
        current_bar = {'volume': df.iloc[0]['volume'], 'average_adr': df.iloc[0]['average_adr'],'timestamp': df.index.to_series()[0], 'Open': df.iloc[0]['Open'], 'High': df.iloc[0]['High'], 'Low': df.iloc[0]['Low'], 'Close': df.iloc[0]['Close']}
        current_high = current_bar['High']
        current_low = current_bar['Low']

        for index, row in df.iterrows():
            high = row['High']
            low = row['Low']
            range_size = row['average_adr'] * 0.1

            if high - current_low >= range_size:
                current_bar['Close'] = current_low + range_size
                range_bars.append(current_bar)

                num_bars = int((high - current_low - range_size) // range_size)
                for i in range(num_bars):
                    current_bar = {'volume': row['volume'], 'average_adr': row['average_adr'],'timestamp': index, 'Open': current_low + range_size * (i), 'High': current_low + range_size * (i + 1), 'Low': current_low + range_size * (i), 'Close': current_low + range_size * (i + 1)}
                    range_bars.append(current_bar)

                current_bar = {'volume': row['volume'] * num_bars, 'average_adr': row['average_adr'],'timestamp': index, 'Open': current_low + range_size * num_bars, 'High': high, 'Low': current_low + range_size * num_bars, 'Close': row['Close']}
                current_high = high
                current_low = current_bar['Low']

            elif current_high - low >= range_size:
                current_bar['Close'] = current_high - range_size
                range_bars.append(current_bar)

                num_bars = int((current_high - low - range_size) // range_size)
                for i in range(num_bars):
                    current_bar = {'volume': row['volume'], 'average_adr': row['average_adr'],'timestamp': index, 'Open': current_high - range_size * (i + 1), 'High': current_high - range_size * (i), 'Low': current_high - range_size * (i + 2), 'Close': current_high - range_size * (i + 1)}
                    range_bars.append(current_bar)

                current_bar = {'volume': row['volume'] * (num_bars + 1), 'average_adr': row['average_adr'],'timestamp': index, 'Open': current_high - range_size * (num_bars + 1), 'High': current_high - range_size * num_bars, 'Low': low, 'Close': row['Close']}
                current_high = current_bar['High']
                current_low = low
            else:
                current_high = max(current_high, high)
                current_low = min(current_low, low)
                current_bar['timestamp'] = index
                current_bar['High'] = current_high
                current_bar['Low'] = current_low
                current_bar['Close'] = row['Close']
                current_bar['average_adr'] = row['average_adr']
                current_bar['volume'] = row['volume']

        return pd.DataFrame(range_bars)

In [ ]:
rb_df = create_range_bar_df(df2.copy())
rb_df['timestamp'] = pd.to_datetime(rb_df['timestamp'])
# can't use set_index because of duplicate timestamps
rb_df

In [ ]:
rb_df = rb_df.sort_values(by='timestamp')
rb_df


add macd, we select 26 for the slow moving average, 12 for the fast moving average and 9 for the signal line.

In [ ]:
# calculate MACD with custom parameters
macd = ta.trend.MACD(rb_df['Close'], window_slow=26, window_fast=12, window_sign=9)
# add MACD values to the dataframe as new columns
rb_df['macd'] = macd.macd()
rb_df['macd_signal'] = macd.macd_signal()
rb_df['macd_histogram'] = macd.macd_diff()
rb_df.dropna(inplace=True)
rb_df


add bb we select 12 for the moving average and 2 for the standard deviation.

In [ ]:
bb = ta.volatility.BollingerBands(rb_df['Close'], window=12, window_dev=2)

# add upper and lower Bollinger Bands to the dataframe as new columns
rb_df['bb_upper'] = bb.bollinger_hband()
rb_df['bb_lower'] = bb.bollinger_lband()
rb_df.dropna(inplace=True)
rb_df


add rsi we use a value of 7 for the RSI.

In [ ]:
from ta.momentum import ROCIndicator
rb_df['roc'] = -1
roc = ROCIndicator(rb_df['Close'], window=14).roc()
rb_df['roc'] = roc
rb_df.head(20)

In [ ]:
rsi = ta.momentum.RSIIndicator(rb_df['Close'], window=7).rsi()
rb_df['rsi'] = rsi
rb_df.dropna(inplace=True)
rb_df.head(20)


In [ ]:
rb_df.reset_index(drop=True, inplace=True)
rb_df['signal'] = 0
rb_df['false_signal'] = -1
rb_df

In [ ]:
def iterations_back_till_condition(series, condition):
    count = 0
    for value in series[::-1]:
        print(f'value: {value}')
        if condition(value):
            print(f'Condition met at index {value}')
            break
        count += 1
    return count

# create example series
s = pd.Series([1, 2, 3, 4, 5])

# pass the equality operator as a lambda function
count = iterations_back_till_condition(s, lambda x: x == 2)

# print the number of iterations before the condition was true
print(f'Number of iterations before condition was true: {count}')

In [ ]:
def mark_false_signals(df_sample: pd.DataFrame, indexes: list, expect_long: bool):
    from scipy.stats import linregress
    # count false signals
    false_signals = 0
    true_signals = 0
    for idx in indexes:
        try:
            seg = list(df_sample[idx:idx+3]['Close'])
            # print(f'long seg: {str(seg)}')
            slope, _, _, _, _ = linregress(range(len(seg)), seg)
            if expect_long:
                if slope < 0:
                    false_signals += 1
                    df_sample.loc[idx, 'false_signal'] = 1
                else:
                    df_sample.loc[idx, 'false_signal'] = 0
                    true_signals += 1
            else:
                if slope > 0:
                    false_signals += 1
                    df_sample.loc[idx, 'false_signal'] = 1
                else:
                    df_sample.loc[idx, 'false_signal'] = 0
                    true_signals += 1            
        except Exception as e:
            print(f'exception: {str(e)}')       
    print(f'false_signals: {false_signals}, true_signals: {true_signals}, diff: {true_signals - false_signals}')     

In [ ]:
class RangeBarStrategy:
    
    long_indexes = []
    short_indexes = []
  
    def __init__(self, df):
        self.df = df
        self.updated_df = pd.DataFrame()

    def bb_upper_near(self, index, row):
        try:
            upper_series = self.df.iloc[:index+1]['bb_upper']
            if len(upper_series) < 2:
                return False
            # print(f'upper_series:\n{str(upper_series)}')
            close = row['Close']
            since = self.iterations_back_till_condition(upper_series, lambda x: x >= close)
            # print(f'bb_upper_near(upper_series >= close): index: {index}, close: {close}, since: {since}')
            return since < 2
        except Exception as e:
            print(f'bb_upper_near: exception: {e.__cause__}')
            raise e
        

    def bb_lower_near(self, index, row):
       try:
         lower_series = self.df.iloc[:index+1]['bb_lower']
         if len(lower_series) < 2:
                return False
        #  print(f'lower_series:\n{str(lower_series)}')
         close = row['Close']
         since = self.iterations_back_till_condition(lower_series, lambda x: x <= close)
        #  print(f'bb_lower_near(lower_series <= close): index: {index}, close: {close}, since: {since}')
         return since < 2
       except Exception as e:
            print(f'bb_lower_near: exception: {e.__cause__}')
            raise e
       
    def iterations_back_till_condition(self, series, condition):
        count = 0
        for value in series[::-1]:
            if condition(value):
                break
            count += 1
        return count
       
    def bb_upper_pointing_up(self, index):
        from scipy.stats import linregress
        bb_seg = self.df.iloc[index-3:index+1]['bb_upper']
        # print(f'bb_seg: {len(bb_seg)}')
        if len(bb_seg) > 0:
            seg_len = len(bb_seg)
            try:
                slope, _, _, _, _ = linregress(range(seg_len), bb_seg)
                # print(f'bb_upper_pointing_up: seg_len: {seg_len}, slope: {slope}')
                return slope > 0
            except Exception as e:
                print(f'bb_upper_pointing_up: exception: {str(e)}')
        return False

    def bb_lower_pointing_down(self, index):
        from scipy.stats import linregress
        bb_seg = self.df.iloc[index-3:index+1]['bb_lower']
        # print(f'bb_seg: {len(bb_seg)}')
        if len(bb_seg) > 0:
            seg_len = len(bb_seg)
            try:
                slope, _, _, _, _ = linregress(range(seg_len), bb_seg)
                # print(f'bb_lower_pointing_down: seg_len: {seg_len}, slope: {slope}')
                return slope < 0
            except Exception as e:
                print(f'bb_lower_pointing_down: exception: {str(e)}')  
           
        return False
    
    def scan(self):
        for index, row in self.df.iterrows():
            try:
                updated_row = self.next(row, index)
                self.df.loc[index] = updated_row
            except Exception as e:
                print(f'next: exception: {str(e)}')

    def next(self, row, index):
        is_long_rsi = row['rsi'] > 70
        is_long_macd = row['macd'] > row['macd_signal'] > 0
        is_bb_upper_near = self.bb_upper_near(index, row)
        is_bb_upper_pointing_up = self.bb_upper_pointing_up(index)

        is_short_rsi = row['rsi'] < 30
        is_short_macd = row['macd'] < row['macd_signal'] < 0
        is_bb_lower_near = self.bb_lower_near(index, row)
        is_bb_lower_pointing_down = self.bb_lower_pointing_down(index)

        # is_not_eq_last_close = row['Close'] != self.df.iloc[index-1]['Close']

        if is_long_rsi and is_long_macd and is_bb_upper_near and is_bb_upper_pointing_up:
            self.long_indexes.append(index)
            row['signal'] = 1
        elif is_short_rsi and is_short_macd and is_bb_lower_near and is_bb_lower_pointing_down:
            row['signal'] = -1
            self.short_indexes.append(index)
        else:
            row['signal'] = 0
        return row       

    def print(self):
        print(f'long_indexes: {self.long_indexes}')
        print(f'short_indexes: {self.short_indexes}')


In [ ]:
df_sample = rb_df
strategy = RangeBarStrategy(df_sample)
strategy.scan()
print(f'long_indexes: {len(strategy.long_indexes)}')
print(f'short_indexes: {len(strategy.short_indexes)}')

## With is_not_eq_last_close
### 3 day average adr
* long_indexes: 1360
* short_indexes: 1600
### 7 day average adr
* long_indexes: 1872
* short_indexes: 2083

### using 3 on near
false_signals: 1697, true_signals: 8707, diff: 7010
false_signals: 1841, true_signals: 9182, diff: 7341
### using 2 on near
false_signals: 1549, true_signals: 7952, diff: 6403
false_signals: 1664, true_signals: 8266, diff: 6602

In [ ]:
mark_false_signals(df_sample, strategy.long_indexes, True)
mark_false_signals(df_sample, strategy.short_indexes, False)
df_sample

In [ ]:
df_sample.to_csv(output_path)

In [ ]:
df_sample.columns

## Plot shows false signals in different color - for purpose of figuring out how to eliminate them

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
from bokeh.layouts import column
from bokeh.models import Range1d

# Create a ColumnDataSource object to hold the plot data
source = ColumnDataSource(data=dict(
    x=df_sample.index,
    y=df_sample['Close'],
    bb_upper=df_sample['bb_upper'],
    bb_lower=df_sample['bb_lower'],
    signal=df_sample['signal'],
    false_signal=df_sample['false_signal'],
    average_adr=df_sample['average_adr'],
    macd=df_sample['macd'],
    macd_signal=df_sample['macd_signal'],
    macd_histogram=df_sample['macd_histogram'],
    rsi=df_sample['rsi']
))

# Create views of the source using BooleanFilters
false_buy_filter = BooleanFilter([True if s==1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
false_sell_filter = BooleanFilter([True if s==-1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
false_buy_view = CDSView(source=source, filters=[false_buy_filter])
false_sell_view = CDSView(source=source, filters=[false_sell_filter])
buy_filter = BooleanFilter([True if s==1 and fs==0 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
sell_filter = BooleanFilter([True if s==-1 and fs==0 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
buy_view = CDSView(source=source, filters=[buy_filter])
sell_view = CDSView(source=source, filters=[sell_filter])

# Create a Range1d object for the x-axis range shared by all three figures
x_range = Range1d(start=df_sample.index[0], end=df_sample.index[-1])

# Create a new figure object
p = figure(title='ss', x_axis_label='Date',
           y_axis_label='Price', width=1400, x_range=x_range)

# Plot the Close values as a line
p.line(x='x', y='y', source=source, legend_label='Close')

# Plot the bb_upper and bb_lower values as shaded areas
p.varea(x='x', y1='bb_upper', y2='bb_lower', source=source, alpha=0.2, legend_label='Bollinger Bands')

# Mark the long signals with green triangles and the short signals with red triangles
p.triangle(x='x', y='y', source=source, view=buy_view, color='green', size=10, legend_label='Buy Signal')
p.inverted_triangle(x='x', y='y', source=source, view=sell_view, color='red', size=10, legend_label='Sell Signal')

p.triangle(x='x', y='y', source=source, view=false_buy_view, color='black', size=10, legend_label='False Buy Signal')
p.inverted_triangle(x='x', y='y', source=source, view=false_sell_view, color='black', size=10, legend_label='False Sell Signal')


p_macd = figure(title='MACD', x_axis_label='Date', y_axis_label='Value', width=1400, x_range=x_range)
# Plot the average ADR as a line
p_macd.line(x='x', y='average_adr', source=source, color='orange', legend_label='Average ADR')

# Plot the MACD as two lines (macd and signal)
p_macd.line(x='x', y='macd', source=source, color='blue', legend_label='MACD')
p_macd.line(x='x', y='macd_signal', source=source, color='red', legend_label='Signal')

# Plot the MACD histogram as a bar chart
p_macd.vbar(x='x', top='macd_histogram', source=source, width=1.0, alpha=0.2, color='gray', legend_label='Histogram')

# Plot the RSI as a line in a separate figure
p_rsi = figure(title='RSI', x_axis_label='Date', y_axis_label='RSI Value', width=1400, x_range=x_range)
p_rsi.line(x='x', y='rsi', source=source, color='purple')

# Show both figures side by side
show(column(p, p_macd, p_rsi))

# Save the plot as an HTML file
output_file('charts/ss_strategy_1min_7d_avg_adr_range_bars_improvements.html')
